In [1]:
import pandas as pd
import numpy as np

In [80]:
year_joined = pd.read_csv('data/year_joined.csv')
emails = pd.read_csv('data/emails.csv', parse_dates=['week'])
donations = pd.read_csv('data/donations.csv')

In [3]:
year_joined

,user,userStats,yearJoined
0,0,silver,2014
1,1,silver,2015
2,2,silver,2016
3,3,bronze,2018
4,4,silver,2018
...,...,...,...
995,995,bronze,2016
996,996,bronze,2018
997,997,bronze,2018
998,998,bronze,2017


In [4]:
emails

,emailsOpened,user,week
0,3.0,1.0,2015-06-29
1,2.0,1.0,2015-07-13
2,2.0,1.0,2015-07-20
3,3.0,1.0,2015-07-27
4,1.0,1.0,2015-08-03
...,...,...,...
25483,3.0,998.0,2018-04-30
25484,3.0,998.0,2018-05-07
25485,3.0,998.0,2018-05-14
25486,3.0,998.0,2018-05-21


In [5]:
year_joined.groupby('user').count().groupby("userStats").count()

,yearJoined
userStats,
1,1000


In [6]:
emails[emails.emailsOpened < 1]

,emailsOpened,user,week


In [7]:
emails[emails.user == 998]

,emailsOpened,user,week
25464,1.0,998.0,2017-12-04
25465,3.0,998.0,2017-12-11
25466,3.0,998.0,2017-12-18
25467,3.0,998.0,2018-01-01
25468,3.0,998.0,2018-01-08
25469,2.0,998.0,2018-01-15
25470,3.0,998.0,2018-01-22
25471,2.0,998.0,2018-01-29
25472,3.0,998.0,2018-02-05
25473,3.0,998.0,2018-02-12


In [8]:
(np.max(emails[emails.user == 998].week) - np.min(emails[emails.user == 998].week)).days / 7

25.0

In [38]:
cutoff_dates = emails.groupby('user').week.agg(['min', 'max']).reset_index()
cutoff_dates

,user,min,max
0,1.0,2015-06-29,2018-05-28
1,3.0,2018-03-05,2018-04-23
2,5.0,2017-06-05,2018-05-28
3,6.0,2016-12-05,2018-05-28
4,9.0,2016-07-18,2018-05-28
...,...,...,...
534,991.0,2016-10-24,2016-10-24
535,992.0,2015-02-09,2015-07-06
536,993.0,2017-09-11,2018-05-28
537,995.0,2016-09-05,2018-05-28


In [39]:
emails.columns

Index(['emailsOpened', 'user', 'week'], dtype='object')

In [40]:
complete_idx = pd.MultiIndex.from_product([np.unique(emails.week), np.unique(emails.user)], names=['week', 'user'])

In [77]:
all_emails = emails.set_index(['week', 'user']) \
    .reindex(complete_idx, fill_value=0) \
    .reset_index()
all_emails.shape

(93247, 3)

In [78]:
for _, row in cutoff_dates.iterrows():
    user = row['user']
    start_date = row['min']
    end_date = row['max']
    
    all_emails.drop(
        all_emails[(all_emails.user == user) & (all_emails.week < start_date)].index,
        inplace=True
    )
    
    all_emails.drop(
        all_emails[(all_emails.user == user) & (all_emails.week > end_date)].index,
        inplace=True
    )

In [81]:
donations.timestamp = pd.to_datetime(donations.timestamp)

In [83]:
donations.set_index('timestamp', inplace=True)

In [99]:
agg_don = donations.groupby('user').apply(
    lambda df: df.amount.resample("W-MON").sum().dropna())
agg_don = agg_don.reset_index()

In [135]:
agg_don[agg_don.user == user]

,user,timestamp,amount
32351,998.0,2018-01-08,50.0


In [146]:
merged_df = pd.merge(all_emails, agg_don, left_on=['user', 'week'], right_on=['user', 'timestamp']) \
    [['user', 'week', 'emailsOpened', 'amount']] \
    .sort_values(['user', 'week'])